## LAB 3.1 Classification of Images Using Transfer Learning

### Import the necessary libraries and functions
Import required libraries and the MNIST dataset, a dataset of handwritten digits often used for training and testing machine learning models.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

Load and unpack the MNIST dataset into training and testing sets for images (x) and labels (y).

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

Convert class labels to one-hot encoded vectors for both training and testing sets.

In [3]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### Load a pretrained MobileNetV2 model without the classification layer
The code initializes a MobileNetV2 model using TensorFlow/Keras. It has an input shape of (224, 224, 3), excludes the top layers for feature extraction, and uses pre-trained ImageNet weights. This makes it suitable for tasks like transfer learning in image classification.

In [4]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


### Add custom layers on top of the pre-trained model
- Using TensorFlow’s Keras API, this code builds a convolutional neural network (CNN). 
- Layers for reshaping, convolution, pooling, flattening, and fully connected operations are included. 
- Dropout is used to achieve regularisation. 
- Using softmax activation, the model, which is ideal for image classification like MNIST, generates class probabilities. 
- The design achieves a compromise between feature extraction and categorization, allowing for successful learning and generalization.

In [5]:
model = models.Sequential()
model.add(layers.Reshape((28, 28, 1), input_shape=(28, 28)))  # Add a channel dimension
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

C:\Users\PRS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\reshaping\reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Freeze the pretrained layers 
                                               
- It is essential that the convolutional base be frozen prior to model compilation and training
-  Freezing ( through layer setting).trainable = False) stops a layer’s weights from changing while it is being trained.
-  Since there are numerous layers in MobileNet V2, all of them will be frozen if the trainable flag for the model is set to False.

In [6]:
for layer in base_model.layers:
    layer.trainable = False

### Compile the model
It is essential to utilize a lower learning rate at this point because you are training a much larger model and want to readjust the pretrained weights. If not, your model may rapidly become overfit.

In [7]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Custom training loop

In [8]:
epochs = 10
batch_size = 32

# Create tf.data.Dataset for training and validation
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training loop
    for images, labels in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(images)
            loss = tf.keras.losses.categorical_crossentropy(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer = optimizers.Adam(learning_rate=0.001)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Validation loop
    accuracy = tf.metrics.CategoricalAccuracy()
    for images, labels in val_dataset:
        predictions = model(images)
        accuracy.update_state(labels, predictions)
    val_acc = accuracy.result().numpy()
    print(f"Validation Accuracy: {val_acc}")


Epoch 1/10
Validation Accuracy: 0.9714000225067139
Epoch 2/10
Validation Accuracy: 0.9710000157356262
Epoch 3/10
Validation Accuracy: 0.9717000126838684
Epoch 4/10
Validation Accuracy: 0.9710000157356262
Epoch 5/10
Validation Accuracy: 0.970300018787384
Epoch 6/10
Validation Accuracy: 0.9708999991416931
Epoch 7/10
Validation Accuracy: 0.9754999876022339
Epoch 8/10
Validation Accuracy: 0.977400004863739
Epoch 9/10
Validation Accuracy: 0.9725000262260437
Epoch 10/10
Validation Accuracy: 0.977400004863739


### Evaluate the model performance on test set

In [9]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9723 - loss: 0.5399
Test Accuracy: 0.977400004863739
